In [1]:
from webapp import tabby
tabby#.servable()

Tabs
    [0] Column(name='Parcel Search')
        [0] Row
            [0] Accordion(name='cities')
                [0] Row(name='filter by city')
                    [0] CheckBoxGroup(options=['all', 'Barrington', ...], value=['all'])
                    [1] CheckBoxGroup(options=['Narragansett', ...])
            [1] Select(name='Presets', options=['', 'stonelink and greenf...])
        [1] Row
            [0] Column
                [0] TextInput(name='Search by Owner')
                [1] Checkbox(name='search all owner c..., value=True)
            [1] TextInput(name='Search by Property L...)
            [2] TextInput(name='Search by Owner Address')
        [2] Row
            [0] Button(name='Update')
            [1] Button(name='Reset')
            [2] TextInput(name='Filename', value='properties.csv')
            [3] FileDownload(auto=False, file='temp/tempprop.csv', filename='properties.csv', label='Transfer properties.csv')
        [3] Tabulator(formatters={'LastSale': DateFormatter...}, frozen_columns=['Location'], pagination='remote', show_index=False, value=              ..., width=1000, widths={'Location': 150, ...})
        [4] TextInput(name='debug')
    [1] Column(name='Group by owner')
        [0] Row
            [0] Column
                [0] Button(name='Update')
                [1] Checkbox(name='Show totals')
                [2] Checkbox(name='Grand Total')
            [1] Column
                [0] TextInput(name='Filename', value='GroupByOwner.csv')
                [1] FileDownload(auto=False, file='temp/tempowners.csv', filename='GroupByOwner.csv', label='Transfer GroupByOwner.csv...)
        [1] Tabulator(formatters={'LastSale': DateFormatter...}, value=Empty DataFrame
Columns: [..., width=1000)
        [2] StaticText(value='Update button g...)
    [2] Column(name='Map Parcels')
        [0] Row
            [0] Button(name='Update Map')
            [1] FileDownload(file='map.html', filename='map.html', label='Download map.html')
            [2] DiscreteSlider(formatter='%d', name='Max number of points', options=[100, 200, 300, ...], value=300)
        [1] Folium(Map, sizing_mode='stretch_both')
        [2] StaticText(value='Update button m...)